In [2]:
!pip -q install langchain-groq duckduckgo-search
!pip -q install -U langchain_community tiktoken langchainhub
!pip -q install -U langchain langgraph tavily-python

In [3]:
!pip show langgraph

Name: langgraph
Version: 0.0.55
Summary: langgraph
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /Users/jcolamendy/python/tutorials/nlp-tutorials/venv/lib/python3.9/site-packages
Requires: langchain-core, uuid6
Required-by: 
